In [92]:
###############
#still working#
###############

import math
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import dataset
import cv2

from tensorflow.examples.tutorials.mnist import input_data

tf.__version__

'1.5.0'

In [93]:
classes = ['dogs', 'cats']
num_classes = len(classes)
 
train_path='/Users/miezai/Downloads/train_200'

data = input_data.read_data_sets('/Users/miezai/Downloads/train_200/', one_hot=True)

validation_size = 0.2
 
batch_size = 16
 
#data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)

Extracting /Users/miezai/Downloads/train_200/train-images-idx3-ubyte.gz
Extracting /Users/miezai/Downloads/train_200/train-labels-idx1-ubyte.gz
Extracting /Users/miezai/Downloads/train_200/t10k-images-idx3-ubyte.gz
Extracting /Users/miezai/Downloads/train_200/t10k-labels-idx1-ubyte.gz


In [94]:
# Convolutional Layer 1.
filter_size1 = 3 
num_filters1 = 32
# Convolutional Layer 2.
filter_size2 = 3
num_filters2 = 32
# Convolutional Layer 3.
filter_size3 = 3
num_filters3 = 64

fc_size = 128            
num_channels = 3
img_size = 128
img_size_flat = img_size * img_size * num_channels
img_shape = (img_size, img_size)

In [95]:
#helpful functions
def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  

    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    biases = create_biases(num_filters)
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')
 
    layer += biases
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    layer = tf.nn.relu(layer)
    return layer

def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
 
    return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer


In [96]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size1,
               num_filters=num_filters1)
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters1,
               conv_filter_size=filter_size2,
               num_filters=num_filters2)
layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters2,
               conv_filter_size=filter_size3,
               num_filters=num_filters3)
layer_flat = create_flatten_layer(layer_conv3)
layer_shape = layer_conv3.get_shape()
num_features = layer_shape[1:4].num_elements()
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=num_features,
                     num_outputs=fc_size,
                     use_relu=True)
layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_size,
                     num_outputs=num_classes,
                     use_relu=False)

In [97]:
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)
